**Table of Contents**
- Mission statment
- Topics-Dynamic User Model
    - Why it does not represent the phanomena we deal with
    - Preferences change over time
- Topics-Dynamic-Reverse User Model
    - Preferences change over time
- Baseline - Vanilla AutoRec
    - Multiple decision rules
    - MRSE
    - ARRI
    - Model Convergence
- AutoRec with ratings history
    - Different temporal window size
    - Different initialization user-item-window tensor
    - MRSE
    - ARRI
    - Model Convergence
- AutoRec with ratings history & recommendations history
    - Different temporal window size
    - Different initialization user-item-window tensor
    - MRSE
    - ARRI
    - Model Convergence
- Topic-Dynamic-Reverse Recommender-Agnostic\
Breaking the recommender-environment feedback loop
- Bonus Section - Repeated Items
    - Topics-Dynamic-Reverse user model
    - Topic-Dynamic-Reverse Recommender-Agnostic environment
    - AutoRec with histories and repeated items
- Conclusion

## Section 1: Mission Statment

Recommending items from a fixed-size set of items to a fixed-size set of users is an open challenge for the following reasons (partial list): 
- **i.i.d principle violation**:\
One of the basic assumptions in training a 'classic' predictive model is independence & identically distribution of the training data. As a consequence, one assumes that all data points presented to the model during training are drawn from a time-invariant distribution.
But in Rec Systems, it does not hold. The users change their behaviors (their underlying distributions over items) as a response to the recommender decisions.
This forces the recommender to be strategic in the face of uncertainty, which is quite difficult during training when the recommender is yet quite incapable in terms of predictive performance.
- **Time-variant distribution**:\
Even regardless of the feedback loop between the recommender & the users, still the user model changes frequently over time during training.
Since the recommender model (at least the vanilla version) does not have a sense of time, it is hardly capable of learning the preferences **changing function**, even when it is user-invariant, time-invariant, and changes slow enough. 
    - We hypothesize that if a model would be capable to learn the users' underlying change function, it could use this information for better predictions, which in turn would lead to better recommendations.
    - We examined various recommender models that were trained on a tensor of observed ratings over last k iterations rather than a single frame-shot rating matrix in the vanilla AutoRec version.
    - Furthermore, we hypothesized that if we will feed the model with its previous decisions (another temporal window of recommendations), it will be able to mitigate the i.i.d violation and the time-variant problems.
    - TLDR: it didn't work though.
    
- For the purpose of this project we used [RecLab](https://berkeley-reclab.github.io), a framework that implements recommendation system environments and recommenders <cite>[1]</cite>. Specifically, we used the Topics environment from class, and the AutoRec recommender; this recommender holds a (2D) item-user table of previous ratings, initialized with zeros and updated whenever a new rating is revealed. This table is very sparse at the start, and its filling ratio depends on the rating frequency (the proportion of active users from all users). At each stage, the Autorec is trained over this table, trying to predict the rating of all item-user pairs given the current table's state.

[1]: https://dl.acm.org/doi/pdf/10.1145/1961634.1961641

## Section 2: Topics-Dynamic User-Model

There are countless user models, aka, ways that the user can change their behavior over time.\
We choose to keep it simple, and stay close as possible to the Topics-Dynamic User-Model we saw in class.\
In this User-Model, all items are grouped into topics, and users prefer topics, rather than individual items.\
The preferences of user $u$ for items $i$ of topic $k$
is initialized as $\pi(u, k) \sim U(0.5, 5.5)$,\
while the topic, $k$ of item $i$ is chosen randomly from the set of all topics.\
When user $u$ is recommended item $i$ it will rate the item as $r_t(u, i)=clip(\pi(u, k)+\epsilon)$ where $\epsilon \sim N(0,\sigma^2)$ represents exogenous noise not,
modeled by the simulation, and clip truncates values to be between $1$ and $5$.\
if at round $t$ user $u$ is recommended an item of topic $k$,\
then at the next round her preferences are updated to be:\
$\pi_{t+1}(u, k) ← clip(\pi_t(u, k) + a),$\
$\pi_{t+1}(u, k') ← clip(\pi_t(u, k') - \frac{a}{K-1}) \quad \forall k' \neq k$\

### Section 2.1: Why it does not demonstrate the phanomena we are intrested

As one could understand from the equation above, this User Model **makes the users like the recommended items.**\
In other words, even if initially the users don't favor the topic of the recommended items, quickly they 'change their minds and start to like it.\
We think that this causes the ARRI for the dynamic environment to raise above the ARRI for the static environment, even without exploration at all, whether the recommender has succeeded to predict the preferences well or not.\
We want to see what happens when the recommendations have an **opposite effect** on the users' behavior.

![TOPICS_DYNAMIC_AUTO_ARRI](project_artifacts/env_d_eg_0_0/ARRI.png)\
**Fig 1:** Autorec with Topics-Dynamic env with change coefficient 0 (Topics-Static) & 1 (Topics-Dynamic with slow drift), averaged over 10 repeats: the centerline is the average, and the vertical lines are error bars (std). While the ARRI for the static version is around 4.0 at best, the ARRI for the dynamic version is around 4.75.

![topics_dynamic_tc0_users_pref](project_artifacts/topics_dynamic_users_pref.jpeg)\
**Fig 2:** User preferences over time for static and dynamic environments. 

## Section 3: Topics-Dynamic Reverse User-Model

- To create an interesting scenario, we modified the User Model source code and created a 'rebelling' version of the user.
    - That is a user which tends to dislike whatever the recommender recommends him.
- The preferences change equations are similar to the previous but with a little change:
    - $\pi_{t+1}(u, k) ← clip(\pi_t(u, k) - a),$
    - $\pi_{t+1}(u, k') ← clip(\pi_t(u, k') + \frac{a}{K-1}) \quad \forall k' \neq k$
- **We would consider this as a baseline for our next experiments.**

![env_dr_eg_0_0__ARRI](project_artifacts/env_dr_eg_0_0/ARRI.png)\
**Fig 3:** Reverse-Topics-Dynamic.\
ARRI for the environment with the preferences change now is lower than the static version.

![REVERSE_TOPICS_DYNAMIC_AUTO_EPS_025_ARRI](project_artifacts/env_dr_eg_0_25/ARRI.png)\
**Fig 4:** Reverse-Topics-Dynamic. Autorec recommender with Epsilon Greedy decision rule (0.25)\
ARRI is not changed much, evidencing that it's not a matter of exploration, but fundamental with the dynamics of the environment.

![topics_dynamic_rev_tc1_users_pref](project_artifacts/topics_dynamic_rev_tc1_users_pref.jpeg)\
**Fig 5:** User preferences change over time for Reverse-Topics-Dynamic\
Pay attention to the collapse in topic 5 in the very first iterations of the simulation

### Section 3.1 - Intermediate observation/hypothesis:

ARRI for the Reverse-Topics-Dynamic is lower than the static version, and roughly fluctuate around the default rating of the Model's users-items-ratings matrix, which is 3.\
We hypothesize that the recommender model does not succeed to learn the preferences **change function** due to a lack of temporal information. Note that the function is time-invariant, user-invariant.\
We think that if we would provide the recommender model with some **temporal information** about the overserved ratings arrived from the users, it would be able to learn the preferences drift, and recommend items accordingly.\
For example, a better prediction using the temporal data, combined with other-than greedy decision rules (epsilon greedy for instance), can help the recommender recommend better items, in the face of lower uncertainty in the future. 

![env_dr_eg_0_0__RMSE](project_artifacts/env_dr_eg_0_0/RMSE.png)\
**Fig 6:** Reverse-Topics-Dynamic RMSE.\
While the model predictive error gets lower for the static version as the simulation continuous, predictive error for the reverse-dynamic version gets higher.

## Section 4 - Vannila Autorec:

As a baseline, we evaluated the regular Autorec recommender (https://arxiv.org/abs/2007.07224) with our new user model, different epsilon greedy values, and averaged over 10 independent experiments.\
**In short, Vanilla Autorec failed to increase its ARRI over the reverse-dynamic-env.**

To show that the problem is not related to the decision rule, we evaluated the follwoing values of epsilon greedy strategy: 0.0, 0.25, 0.5.\
TL;DR: this is not the problem.

<div align="center">
<table>
    <tr>
<!--     <div align="center"> -->
    <td>Epsilon Greedy = 0.00</td>
<!--     </div> -->
<!--     <div align="center"> -->
    <td>Epsilon Greedy = 0.25</td>
<!--     </div> -->
<!--     <div align="center"> -->
    <td>Epsilon Greedy = 0.50</td>
<!--     </div> -->
    </tr>
    <tr>
        <td><img src="project_artifacts/env_dr_eg_0_0/ARRI.png" width=3000></td>
        <td><img src="project_artifacts/env_dr_eg_0_25/ARRI.png" width=3000></td>
        <td><img src="project_artifacts/env_dr_eg_0_5/ARRI.png" width=3000></td>
    </tr>
</table>
</div>

**Fig 7:** ARRI with epsilon greedy 0, 0.25, 0.5 (left to right).
While as expected, more exploration adds more variability to the results (higher std), it does not improve the overall expected ARRI.

We leave other-than epsilon greedy decision rules to future research. Most noteably, we think that second-best decision rules might work better than rules from the greedy family for the Reverse-Topics-Dynamic user model.

<div align="center">
<table>
    <tr>
<!--     <div align="center"> -->
    <td>Epsilon Greedy = 0.00</td>
<!--     </div> -->
<!--     <div align="center"> -->
    <td>Epsilon Greedy = 0.25</td>
<!--     </div> -->
<!--     <div align="center"> -->
    <td>Epsilon Greedy = 0.50</td>
<!--     </div> -->
    </tr>
    <tr>
        <td><img src="project_artifacts/env_dr_eg_0_0/RMSE.png" width=3000></td>
        <td><img src="project_artifacts/env_dr_eg_0_25/RMSE.png" width=3000></td>
        <td><img src="project_artifacts/env_dr_eg_0_5/RMSE.png" width=3000></td>
    </tr>
</table>
</div>

**Fig 8:** RMSE with epsilon greedy 0, 0.25, 0.5 (left to right).\
In all cases, in contrast to the static version of the env, RMSE (predictive error) for the reverse dynamic env gets higher. 

To assure that it is not a problem of the model's capacity to learn the users-items-ratings matrix, or any other possible failures of the AutoEncoder Neural net model to learn, we show the training loss of the model, for the last iteration of the simulation.


<div align="center">
<table>
    <tr>
<!--     <div align="center"> -->
    <td>Epsilon Greedy = 0.00</td>
<!--     </div> -->
<!--     <div align="center"> -->
    <td>Epsilon Greedy = 0.25</td>
<!--     </div> -->
<!--     <div align="center"> -->
    <td>Epsilon Greedy = 0.50</td>
<!--     </div> -->
    </tr>
    <tr>
        <td><img src="project_artifacts/env_dr_eg_0_0/REC_LAST_STEP_LOSSES.png" width=3000></td>
        <td><img src="project_artifacts/env_dr_eg_0_25/REC_LAST_STEP_LOSSES.png" width=3000></td>
        <td><img src="project_artifacts/env_dr_eg_0_5/REC_LAST_STEP_LOSSES.png" width=3000></td>
    </tr>
</table>
</div>

**Fig 9:** Last iteration of the simulation training loss.\
These plots show that the model is capable to learn the users-items-ratings matrix (from capacity perspective or optimization algorithm).

## Section 5 - Autorec with Ratings History:

- As mentioned earlier, we hypothesized that the lack of temporal information about the preferences-change function (again, time-invariant & user-invariant) is the cause for the recommender incapability to improve ARRI when interacting with a negative-feedback user model (such as our famous Reverse-Topics-Dynamic environment).
- Based on this assumption we created a new Recommender model (Autorec-based).
- the Vanilla Autorec is fed by the **sparse** users-items-ratings matrix. It is being trained on the same matrix, which is getting denser, as the simulation continues (the model receives more and mode ratings, depends how many active users there are).
- Instead, we created a **Tensor** (3D matrix) of observed ratings of shape **\<window size\> X \<num users\> X \<num items\>**.
- At each time step, a new rating per user is pushed into its column of the tensor.\
In contrast to the vanilla matrix, in which new ratings replaced the old ones (in a user-item cell of the 2D matrix), now we have a user-item <window_size> vector.
- We manage all vectors as queues. The newest values are at the beginning of the queue. 
- All users-items vectors are independent: at a given simulation cycle, if a specific user was not active, all of his per-item vectors don't change.\
Corollary, for an active user, only the vector of the recommended item is changing.
- We hypothesized that the recommender might consider an empty cell (initialized with 0), as rating 0, in the first couple of hundreds of iterations, before all vectors (with high probability) were filled with observed ratings. 
- Hence, we tried random initialization as well. It didn't change the averaged results though.
- We evaluated various window sizes 

<div align="center">
<table>
    <tr>
<!--     <div align="center"> -->
    <td>Window Size = 1</td>
<!--     </div> -->
<!--     <div align="center"> -->
    <td>Window Size = 3</td>
<!--     </div> -->
<!--     <div align="center"> -->
    <td>Window Size = 5</td>
<!--     </div> -->
    </tr>
    <tr>
        <td><img src="project_artifacts/env_dr_rec_ta_1/ARRI.png" width=3000></td>
        <td><img src="project_artifacts/env_dr_rec_ta_3/ARRI.png" width=3000></td>
        <td><img src="project_artifacts/env_dr_rec_ta_5/ARRI.png" width=3000></td>
    </tr>
</table>
</div>

**Fig 10:** Temporal Autorec with different window sizes ARRI.

As you can see, it is not working, and even fails to shift from the default predicted rating, which is 3. 
This might indicate a convergence problem of the model, or something more fundamental. 
Worth mentioning that the std is also much higher compared to the vanilla version.

To figure out what went wrong, we examined to model convergence and prediction error:

<div align="center">
<table>
    <tr>
<!--     <div align="center"> -->
    <td>Window Size = 1</td>
<!--     </div> -->
<!--     <div align="center"> -->
    <td>Window Size = 3</td>
<!--     </div> -->
<!--     <div align="center"> -->
    <td>Window Size = 5</td>
<!--     </div> -->
    </tr>
    <tr>
        <td><img src="project_artifacts/env_dr_rec_ta_1/RMSE.png" width=3000></td>
        <td><img src="project_artifacts/env_dr_rec_ta_3/RMSE.png" width=3000></td>
        <td><img src="project_artifacts/env_dr_rec_ta_5/RMSE.png" width=3000></td>
    </tr>
</table>
</div>

**Fig 11:** Temporal Autorec with different window sizes RMSE.

In terms of RMSE, the Temporal model behaves very similar to the Vanilla flavor, but again, much noisier.

<div align="center">
<table>
    <tr>
<!--     <div align="center"> -->
    <td>Window Size = 1</td>
<!--     </div> -->
<!--     <div align="center"> -->
    <td>Window Size = 3</td>
<!--     </div> -->
<!--     <div align="center"> -->
    <td>Window Size = 5</td>
<!--     </div> -->
    </tr>
    <tr>
        <td><img src="project_artifacts/env_dr_rec_ta_1/REC_LAST_STEP_LOSSES.png" width=3000></td>
        <td><img src="project_artifacts/env_dr_rec_ta_3/REC_LAST_STEP_LOSSES.png" width=3000></td>
        <td><img src="project_artifacts/env_dr_rec_ta_5/REC_LAST_STEP_LOSSES.png" width=3000></td>
    </tr>
</table>
</div>

**Fig 12:** Temporal Autorec with different window sizes Last simulation iteration model convergence.

You can see that the model has converged very nicely.

## Section 6 - Autorec with Ratings History and Past Recommendations History:

- Following the finding in the previous section, we assumed that the problem is not entirely dependent on the lack of temporal information.
- Although time-invariant, and user-invariant, the preferences-change function is not a function of the topic solely, but the recommended item as well.\
$\pi_{t+1}(user, item)=\pi_t(u, k, r_t)$
- Therefore, we assumed that the history of past recommendations, per user, per item, is also needed for the recommender to learn the function.
- We combined the section 6 method with another 2D matrix. This matrix has <num users> rows X <window size> cols. each cell per user contains the past recommendation for that user.
- Similarly, each vector per user is a queue and managed independently. 
- Again, we examined different initialization methods, with little to no difference
- In contrast to section 6, now we have 2 streams of inputs for the Autorec neural network model.
- Each stream goes through a different Fully Connected layer to create its embedding, which in theory embed temporal information based on the last <time window> observations. 
- Then the two embeddings are combined into a single matrix, similar to the vanilla Autorec architecture.

<div align="center">
<table>
    <tr>
<!--     <div align="center"> -->
    <td>Window Size = 1</td>
<!--     </div> -->
<!--     <div align="center"> -->
    <td>Window Size = 3</td>
<!--     </div> -->
<!--     <div align="center"> -->
    <td>Window Size = 5</td>
<!--     </div> -->
    </tr>
    <tr>
        <td><img src="project_artifacts/env_dr_rec_ta2_1/ARRI.png" width=3000></td>
        <td><img src="project_artifacts/env_dr_rec_ta2_3/ARRI.png" width=3000></td>
        <td><img src="project_artifacts/env_dr_rec_ta2_5/ARRI.png" width=3000></td>
    </tr>
</table>
</div>

**Fig 13:** Temporal Autorec with 2 streams with different window sizes ARRI.
    
From the ARRI perspective, there is not much change compared to the single-stream Temporal Autorec version. It is still very noisy, and still, fluctuates around the default predicted rating of 3.
    
<div align="center">
<table>
    <tr>
<!--     <div align="center"> -->
    <td>Window Size = 1</td>
<!--     </div> -->
<!--     <div align="center"> -->
    <td>Window Size = 3</td>
<!--     </div> -->
<!--     <div align="center"> -->
    <td>Window Size = 5</td>
<!--     </div> -->
    </tr>
    <tr>
        <td><img src="project_artifacts/env_dr_rec_ta2_1/RMSE.png" width=3000></td>
        <td><img src="project_artifacts/env_dr_rec_ta2_3/RMSE.png" width=3000></td>
        <td><img src="project_artifacts/env_dr_rec_ta2_5/RMSE.png" width=3000></td>
    </tr>
</table>
</div>

**Fig 14:** Temporal Autorec with 2 streams with different window sizes RMSE.
    
From RMSE (predictive error) perspective, there is a difference. RMSE here is much worse, also for the static user model.\
It might indicate a convergence problem of the model, so please follow us to the next section in our journey to find the absolute truth!
    
<div align="center">
<table>
    <tr>
<!--     <div align="center"> -->
    <td>Window Size = 1</td>
<!--     </div> -->
<!--     <div align="center"> -->
    <td>Window Size = 3</td>
<!--     </div> -->
<!--     <div align="center"> -->
    <td>Window Size = 5</td>
<!--     </div> -->
    </tr>
    <tr>
        <td><img src="project_artifacts/env_dr_rec_ta2_1/REC_LAST_STEP_LOSSES.png" width=3000></td>
        <td><img src="project_artifacts/env_dr_rec_ta2_3/REC_LAST_STEP_LOSSES.png" width=3000></td>
        <td><img src="project_artifacts/env_dr_rec_ta2_5/REC_LAST_STEP_LOSSES.png" width=3000></td>
    </tr>
</table>
</div>

**Fig 15:** Temporal Autorec with 2 streams with different window sizes last iteration model convergence.
    
Here we see that the model succeeded to converge, roughly no less well than the previous models.\
So the question remains: what causes the poor performance?
    
    
### At this point, we assume the problem is more fundamental than the lack of temporal information for the recommender.
### In the next section, we will examine this assumption, using another custom-made user model we created - the Topic-Dynamic-Reverse Rec-Agnostic!

## Section 7 - Topic-Dynamic-Reverse Recommender-Agnostic:

We wondered whether the negative impact on the ARRI is due to the poor predictive performance, or is it because of some negative effect of the recommender-environment feedback loop.\
As we spoke in class, the recommender decisions affect the data distribution, breaking the i.i.d assumption in the basis of all predictive models, hence can cause weird behaviors.\
To check and isolate this effect, we modified the Topics-Dynamics source code, to change randomly regardless of the recommender decisions.\
So we still have time-invariant, user-invariant preferences change function, but now it is recommender-invariant as well.\
For each user, at reset, a single topic is chosen randomly. Then, each time this user receives a recommendation, regardless of the recommended item, the chosen topic preference is being reduced, and the rest are being increased (like regular Reverse-Topic-dynamic).
We repeated the previous experiments with the new environment to see if the source for the reduced performance is a lack of temporal information (as previously assumed), or the feedback loop itself.

<div align="center">
<table>
    <tr>
<!--     <div align="center"> -->
    <td>Epsilon Greedy = 0.00</td>
<!--     </div> -->
<!--     <div align="center"> -->
    <td>Epsilon Greedy = 0.25</td>
<!--     </div> -->
<!--     <div align="center"> -->
    <td>Epsilon Greedy = 0.50</td>
<!--     </div> -->
    </tr>
    <tr>
        <td><img src="project_artifacts/env_drra_eg_0_0/ARRI.png" width=3000></td>
        <td><img src="project_artifacts/env_drra_eg_0_25/ARRI.png" width=3000></td>
        <td><img src="project_artifacts/env_drra_eg_0_5/ARRI.png" width=3000></td>
    </tr>
</table>
</div>

**Fig 16:** Reverse-Topic-Dynamic Recommendation-Agnostic ARRI.\
We observe that the recommender succeeds (kind of) to learn the preferences change function. ARRI for the dynamic version converges with the ARRI for the static version.

<div align="center">
<table>
    <tr>
<!--     <div align="center"> -->
    <td>Epsilon Greedy = 0.00</td>
<!--     </div> -->
<!--     <div align="center"> -->
    <td>Epsilon Greedy = 0.25</td>
<!--     </div> -->
<!--     <div align="center"> -->
    <td>Epsilon Greedy = 0.50</td>
<!--     </div> -->
    </tr>
    <tr>
        <td><img src="project_artifacts/env_drra_eg_0_0/RMSE.png" width=3000></td>
        <td><img src="project_artifacts/env_drra_eg_0_25/RMSE.png" width=3000></td>
        <td><img src="project_artifacts/env_drra_eg_0_5/RMSE.png" width=3000></td>
    </tr>
</table>
</div>

**Fig 17:** Reverse-Topic-Dynamic Recommendation-Agnostic RMSE.\
Also from a predictive error perspective: RMSE drops as the simulation continues.\
This is in contrast to the equivalent environment with a recommender-environment feedback loop (see sections 4-6)

<div align="center">
<table>
    <tr>
<!--     <div align="center"> -->
    <td>Epsilon Greedy = 0.00</td>
<!--     </div> -->
<!--     <div align="center"> -->
    <td>Epsilon Greedy = 0.25</td>
<!--     </div> -->
<!--     <div align="center"> -->
    <td>Epsilon Greedy = 0.50</td>
<!--     </div> -->
    </tr>
    <tr>
        <td><img src="project_artifacts/env_drra_eg_0_0/REC_LAST_STEP_LOSSES.png" width=3000></td>
        <td><img src="project_artifacts/env_drra_eg_0_25/REC_LAST_STEP_LOSSES.png" width=3000></td>
        <td><img src="project_artifacts/env_drra_eg_0_5/REC_LAST_STEP_LOSSES.png" width=3000></td>
    </tr>
</table>
</div>

**Fig 18:** Reverse-Topic-Dynamic Recommendation-Agnostic Last iteration loss.

## Section 8 - Bonus - Repeated Recommended Items:

We thought about it and came to the conclusion that our Temporal Recommender (AutoRec with history), could be better when the same items were recommended repeatedly multiple times.\
The idea was to let the recommender see the same item per user multiple times, and by doing so, fulfilling the history queues and learning the temporal change.
The Basic Topics enviroment does not allow that: If the same Item is recommended more then once to the same user, an exception occures.\
We guess this is why the course stuff choose 100 users x 100 items x 100 steps.\
Anyway, We thought it will be intersting to see if the recommender performance improved when the ratings queue (per user, per item columns in the history tensor) gets full, and then the recommender keep training with full queues. **TL;DR: it doesn't** (surprised?).\
For that, we run the same experinments from sections 5,6,7 with 1000 steps, istead of 100 as before. Since we picked 0.1 active user at each simulation step, on average, each item per user would be presented with probability 1.0.\
And of-course, some items are presented more than once, causing the queues to fill.
<table>
    <tr><td>ARRI</td></tr>
    <tr><td>Window Size = 1</td>
        <td>Window Size = 3</td>
        <td>Window Size = 5</td></tr>
    <tr><td><img src="project_artifacts/steps1000/env_dr_rec_ta_1/ARRI.png" width=3000></td>
        <td><img src="project_artifacts/steps1000/env_dr_rec_ta_3/ARRI.png" width=3000></td>
        <td><img src="project_artifacts/steps1000/env_dr_rec_ta_5/ARRI.png" width=3000></td></tr>
    <tr><td>RMSE</td></tr>
    <tr><td>Window Size = 1</td>
        <td>Window Size = 3</td>
        <td>Window Size = 5</td></tr>
    <tr><td><img src="project_artifacts/steps1000/env_dr_rec_ta_1/RMSE.png" width=3000></td>
        <td><img src="project_artifacts/steps1000/env_dr_rec_ta_3/RMSE.png" width=3000></td>
        <td><img src="project_artifacts/steps1000/env_dr_rec_ta_5/RMSE.png" width=3000></td></tr>
    <tr><td>LOSS</td></tr>
    <tr><td>Window Size = 1</td>
        <td>Window Size = 3</td>
        <td>Window Size = 5</td></tr>
    <tr><td><img src="project_artifacts/steps1000/env_dr_rec_ta_1/REC_LAST_STEP_LOSSES.png" width=3000></td>
        <td><img src="project_artifacts/steps1000/env_dr_rec_ta_3/REC_LAST_STEP_LOSSES.png" width=3000></td>
        <td><img src="project_artifacts/steps1000/env_dr_rec_ta_5/REC_LAST_STEP_LOSSES.png" width=3000></td></tr>
</table>

**Fig 19:** Reverse-Topic-Dynamic, AutoRec with rating history, 1000 steps (items repeat).

<table>
    <tr><td>ARRI</td></tr>
    <tr><td>Window Size = 1</td>
        <td>Window Size = 3</td>
        <td>Window Size = 5</td></tr>
    <tr><td><img src="project_artifacts/steps1000/env_dr_rec_ta2_1/ARRI.png" width=3000></td>
        <td><img src="project_artifacts/steps1000/env_dr_rec_ta2_3/ARRI.png" width=3000></td>
        <td><img src="project_artifacts/steps1000/env_dr_rec_ta2_5/ARRI.png" width=3000></td></tr>
    <tr><td>RMSE</td></tr>
    <tr><td>Window Size = 1</td>
        <td>Window Size = 3</td>
        <td>Window Size = 5</td></tr>
    <tr><td><img src="project_artifacts/steps1000/env_dr_rec_ta2_1/RMSE.png" width=3000></td>
        <td><img src="project_artifacts/steps1000/env_dr_rec_ta2_3/RMSE.png" width=3000></td>
        <td><img src="project_artifacts/steps1000/env_dr_rec_ta2_5/RMSE.png" width=3000></td></tr>
    <tr><td>LOSS</td></tr>
    <tr><td>Window Size = 1</td>
        <td>Window Size = 3</td>
        <td>Window Size = 5</td></tr>
    <tr><td><img src="project_artifacts/steps1000/env_dr_rec_ta2_1/REC_LAST_STEP_LOSSES.png" width=3000></td>
        <td><img src="project_artifacts/steps1000/env_dr_rec_ta2_3/REC_LAST_STEP_LOSSES.png" width=3000></td>
        <td><img src="project_artifacts/steps1000/env_dr_rec_ta2_5/REC_LAST_STEP_LOSSES.png" width=3000></td></tr>
</table>

**Fig 20:** Reverse-Topic-Dynamic, AutoRec with rating history **and recommendations history**, 1000 steps (items repeat).

As you can see, this setting in Reverse-Topic-Dynamic didn't result in any improvements. The general trend is the opposite: compared to the 100 steps Reverse-Topic-Dynamic, which had ARRI around 3, now after ~800 steps it starts to get down below, and reaches ~2.5. It is the same for AutoRec with rating history only and with ratings history + recs history.\
We think that this is because the recommender failed to generalize for the same item, a user over time. In other words, in the later visits on the same item, user tuple, the recommender use outdated knowledge. 

<table>
    <tr><td>ARRI</td></tr>
    <tr><td>Window Size = 1</td>
        <td>Window Size = 3</td>
        <td>Window Size = 5</td></tr>
    <tr><td><img src="project_artifacts/steps1000/env_drra_rec_ta_1/ARRI.png" width=3000></td>
        <td><img src="project_artifacts/steps1000/env_drra_rec_ta_3/ARRI.png" width=3000></td>
        <td><img src="project_artifacts/steps1000/env_drra_rec_ta_5/ARRI.png" width=3000></td></tr>
    <tr><td>RMSE</td></tr>
    <tr><td>Window Size = 1</td>
        <td>Window Size = 3</td>
        <td>Window Size = 5</td></tr>
    <tr><td><img src="project_artifacts/steps1000/env_drra_rec_ta_1/RMSE.png" width=3000></td>
        <td><img src="project_artifacts/steps1000/env_drra_rec_ta_3/RMSE.png" width=3000></td>
        <td><img src="project_artifacts/steps1000/env_drra_rec_ta_5/RMSE.png" width=3000></td></tr>
    <tr><td>LOSS</td></tr>
    <tr><td>Window Size = 1</td>
        <td>Window Size = 3</td>
        <td>Window Size = 5</td></tr>
    <tr><td><img src="project_artifacts/steps1000/env_drra_rec_ta_1/REC_LAST_STEP_LOSSES.png" width=3000></td>
        <td><img src="project_artifacts/steps1000/env_drra_rec_ta_3/REC_LAST_STEP_LOSSES.png" width=3000></td>
        <td><img src="project_artifacts/steps1000/env_drra_rec_ta_5/REC_LAST_STEP_LOSSES.png" width=3000></td></tr>
</table>

**Fig 21:** Reverse-Topic-Dynamic-Recommender-Agnostic, AutoRec with rating history, 1000 steps (items repeat).

<table>
    <tr><td>ARRI</td></tr>
    <tr><td>Window Size = 1</td>
        <td>Window Size = 3</td>
        <td>Window Size = 5</td></tr>
    <tr><td><img src="project_artifacts/steps1000/env_drra_rec_ta2_1/ARRI.png" width=3000></td>
        <td><img src="project_artifacts/steps1000/env_drra_rec_ta2_3/ARRI.png" width=3000></td>
        <td><img src="project_artifacts/steps1000/env_drra_rec_ta2_5/ARRI.png" width=3000></td></tr>
    <tr><td>RMSE</td></tr>
    <tr><td>Window Size = 1</td>
        <td>Window Size = 3</td>
        <td>Window Size = 5</td></tr>
    <tr><td><img src="project_artifacts/steps1000/env_drra_rec_ta2_1/RMSE.png" width=3000></td>
        <td><img src="project_artifacts/steps1000/env_drra_rec_ta2_3/RMSE.png" width=3000></td>
        <td><img src="project_artifacts/steps1000/env_drra_rec_ta2_5/RMSE.png" width=3000></td></tr>
    <tr><td>LOSS</td></tr>
    <tr><td>Window Size = 1</td>
        <td>Window Size = 3</td>
        <td>Window Size = 5</td></tr>
    <tr><td><img src="project_artifacts/steps1000/env_drra_rec_ta2_1/REC_LAST_STEP_LOSSES.png" width=3000></td>
        <td><img src="project_artifacts/steps1000/env_drra_rec_ta2_3/REC_LAST_STEP_LOSSES.png" width=3000></td>
        <td><img src="project_artifacts/steps1000/env_drra_rec_ta2_5/REC_LAST_STEP_LOSSES.png" width=3000></td></tr>
</table>

**Fig 22:** Reverse-Topic-Dynamic-Recommender-Agnostic, AutoRec with rating history **and recommendations history**, 1000 steps (items repeat).

in section 7, Reverse-Topic-Dynamic-Recommender-Agnostic showed pretty promising results. Now, with 1000 steps, even that is not true. The results here are much worse than the 100 steps experiment, and the reasons, we think, are the same as Fig19+20. Anyway, further research is needed to be sure. 

## Section 9 - Finally - Conclusion:

In this project, we explored the world of a 'reverse-dynamic' user environment. Given a 'rebelling' user model, we showed why the standard recommender fails to improve its ARRI over time and hypothesizes the reasons for it. Our idea of improving it with temporal data didn't go well and is a subject for future investigations. In particular, in this project, we focused on the recommender backbone model, rather than the decision rule. It is indeed possible that under our assumptions and use case, this rule needs an update, in a sense that maybe recommending the highest predicted rating is not necessarily the optimal recommendation. Research of this hypothesis may focus on the decision policy itself, perhaps learning it along with the ratings. Moreover, it is possible to check other models; the model maybe failed to generalize between users/items/topics. Future research may try to design another NN which gathers information other than the CF matrix form.

To conclude, we believe the demonstration we provided may lead to an area with a great potential for innovation, which we hope will be achieved in the future to help us better understand the endless possible user models in changing environments. 